# use gdax-python as API to gdax data

How to gather current and historical data and work with time intervals.

## resources

https://github.com/danpaquin/gdax-python

install from github as pypi version is lagging (?).

pip install https://github.com/danpaquin/gdax-python/zipball/master

### GDAX notes

GDAX requests limit: 3 per second

Timepoints: ISO 8601

GDAX private and trading requires Authenticated keys

https://support.gdax.com/customer/en/portal/articles/2425383-how-can-i-create-an-api-key-for-gdax-

The market data does not.

https://docs.gdax.com/?python#market-data

https://docs.gdax.com/?python#get-historic-rates

200 'candles' per request. open high low volume.

24 hour data:

GET /products/<product-id>/stats

websocket feed. rate limit 1 per 4 seconds.

wss://ws-feed.gdax.com

## working with gdax-python api and gdax public marketplace data.

### viewing products from gdax

In [1]:
import gdax

In [6]:
public_client = gdax.PublicClient()

In [ ]:
products = public_client.get_products()

gdax products appear to be limited to the same coins as coinbase. 

They include pricing in british pound and euro.

We want to know only USD products.

In [146]:
import re

In [147]:
GU = re.compile('EUR|GBP')
products = [p for p in products if not GU.search(p['id'])]
display(products[0:2])

[{'base_currency': 'BCH',
  'base_max_size': '250',
  'base_min_size': '0.0001',
  'display_name': 'BCH/USD',
  'id': 'BCH-USD',
  'margin_enabled': False,
  'quote_currency': 'USD',
  'quote_increment': '0.01',
  'status': 'online',
  'status_message': None},
 {'base_currency': 'BCH',
  'base_max_size': '250',
  'base_min_size': '0.0001',
  'display_name': 'BCH/BTC',
  'id': 'BCH-BTC',
  'margin_enabled': False,
  'quote_currency': 'BTC',
  'quote_increment': '0.00001',
  'status': 'online',
  'status_message': None}]

We see that there are both Coin-USD and Coin-Coin producst with some info on (probably) minimum and maximum allowable order size?

Products are one way.

There is BCH-USD, but not USD-BCH.

### Get current ticker containing ask and bid.

In [54]:
public_client.get_product_ticker(product_id='ETH-USD')

{'ask': '835.25',
 'bid': '835.24',
 'price': '835.24000000',
 'size': '0.00179589',
 'time': '2017-12-21T08:47:10.932000Z',
 'trade_id': 22769836,
 'volume': '221867.97204692'}

**TODO: understand the order side.**

https://docs.gdax.com/#get-product-ticker

    SIDE

    The trade side indicates the maker order side. The maker order is the order that was open on the order book. buy side indicates a down-tick because the maker was a buy order and their order was removed. Conversely, sell side indicates an up-tick.


In [55]:
public_client.get_product_trades(product_id='ETH-USD')

[{'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:45.736Z',
  'trade_id': 22769933},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:45.485Z',
  'trade_id': 22769932},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:45.445Z',
  'trade_id': 22769931},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:45.352Z',
  'trade_id': 22769930},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:44.938Z',
  'trade_id': 22769929},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:44.419Z',
  'trade_id': 22769928},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.00179589',
  'time': '2017-12-21T08:47:44.37Z',
  'trade_id': 22769927},
 {'price': '835.24000000',
  'side': 'buy',
  'size': '0.47145985',
  'time': '2017-

The ticker.

    {'ask': '835.25',
     'bid': '835.24',
     'price': '835.24000000',
     'size': '0.00179589',
     'time': '2017-12-21T08:47:10.932000Z',
     'trade_id': 22769836,
     'volume': '221867.97204692'}

Resulted in a trade in the 'recent history' soon afterwards.

     {'price': '835.24000000',
      'side': 'buy',
      'size': '0.00179589',
      'time': '2017-12-21T08:47:10.932Z',
      'trade_id': 22769836}

TODO: confirm this interpretation.

Someone wanted to buy it at 835.24, someone wanted to sell it at 835.25. The price of the trade was 835.24. So it was a buy at 835.24. The order maker sold at 835.24 allowing someone to buy at 835.24.

### get historic data and understand granularity and converting timestamps.

Note the bucket size may be different on each call.

https://docs.gdax.com/#get-historic-rates
    
Each bucket is an array of the following information:

- time : bucket start time (note bucket goes to next time
- low : lowest price during the bucket interval
- high : highest price during the bucket interval
- open : opening price (first trade) in the bucket interval
- close : closing price (last trade) in the bucket interval
- volume : volume of trading activity during the bucket interval

In [103]:
eth_usd_hist = public_client.get_product_historic_rates('ETH-USD')

In [104]:
len(eth_usd_hist)

311

In [69]:
eth_usd_hist[0:3]

[[1513847220, 837.97, 837.98, 837.97, 837.98, 25.93066695],
 [1513847160, 837.97, 837.98, 837.98, 837.97, 30.062845900000003],
 [1513847100, 835.99, 837.98, 836, 837.97, 267.8209014899999]]

In [ ]:
public_client.get_product_historic_rates

In [74]:
import datetime

In [93]:
t = datetime.datetime.fromtimestamp(int("1513847160"))
t

datetime.datetime(2017, 12, 21, 1, 6)

These are one minute time intervals:

    1513847100 : 2017-12-21 1:05
    1513847220 : 2017-12-21 1:07

In [87]:
t.date()
t.time()

datetime.time(1, 6, 10)

In [114]:
datetime.datetime.fromtimestamp(eth_usd_hist[-1][0]).isoformat()

'2017-12-20T20:20:00'

In [113]:
datetime.datetime.fromtimestamp(eth_usd_hist[0][0]).isoformat()

'2017-12-21T01:30:00'

This history spans from 10:10 pm yesterday to 1:30 am today. Approx. 5 hours.

**To get particular time every X minutes, use a combination of start stop and granularity.**

granularity is bucket size in seconds.

No more than 200 intervals are accepted.

In [400]:
# 12-18 from 0 to 24 in 1 hour (3600 second) chunks.

# PST
start = '2017-12-13T08:00:00-8:00'
end = '2017-12-17T02:00:00-08:00'
granularity = 86400 # seconds per bucket, i.e. 1 hour intervals
#btc_usd_hist = public_client.get_product_historic_rates('BTC-USD', end=end, granularity=granularity)
btc_usd_hist = public_client.get_product_historic_rates('BTC-USD', end=end, granularity=86400)

This is seeming kind of messed up for doing anything specific.

Getting different return types depending on whether I include end and start or granularity.

In [401]:
print(len(btc_usd_hist))
for h in btc_usd_hist:
    print(datetime.datetime.fromtimestamp(h[0]).isoformat())

3
2017-12-20T16:00:00
2017-12-18T16:00:00
2017-12-17T16:00:00


In [390]:
public_client.get_product_historic_rates

[[1513814400, 16851, 17158.34, 16852, 16949.99, 660.1336242899963],
 [1513641600, 17763.48, 19099, 19039.01, 18225.47, 11347.418107781108],
 [1513555200, 18425, 19367.5, 18545.02, 19039.01, 17253.83669920245]]

### get historic data: daily stat
Probably the last 24 hours through 'now'.

No option to select other days.

In [71]:
public_client.get_product_24hr_stats('ETH-USD')

{'high': '844.27000000',
 'last': '838.09000000',
 'low': '775.90000000',
 'open': '800.04000000',
 'volume': '220278.79144908',
 'volume_30day': '9535008.28345899'}